In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataset import kline_data
from dateutil import parser

In [2]:
def volume_profile(df, n_bins=10):
    df = df[['open', 'high', 'low', 'close', 'volume']]
    high = np.max(df['high'])
    low = np.min(df['low'])
    d_hl = high - low
    d = d_hl / n_bins
    coords = np.arange(low, high + 0.1, d)
    profile = pd.DataFrame(columns=['price', 'buy_volume', 'sell_volume', 'total_volume'])
    profile['price'] = pd.Series(coords).rolling(2).mean().dropna()
    buy_candles = df.where(df['close'] - df['open'] >= 0).dropna()
    sell_candles = df.where(df['close'] - df['open'] < 0).dropna()
    profile['buy_volume'] = np.histogram(buy_candles['close'], bins=coords, weights=buy_candles['volume'])[0]
    profile['sell_volume'] = np.histogram(sell_candles['close'], bins=coords, weights=sell_candles['volume'])[0]
    profile['total_volume'] = profile['buy_volume'] + profile['sell_volume']
    return profile

In [3]:
def get_daily_volume_profile(data, date, n_bins=10, interval='1m'):
    idx_start = data[data['open_time'] == str(parser.parse(date))].index.values[0]
    idx_end = data[data['open_time'] == str(parser.parse(date) + datetime.timedelta(days=1))].index.values[0]
    day_data = data.iloc[idx_start:idx_end]
    profile = volume_profile(day_data, n_bins=n_bins)
    return profile

In [4]:
def plot_volume_profile(profile):
    plt.figure()
    plt.bar(profile['price'], profile['buy_volume'],
            width=profile['price'].iloc[-1] - profile['price'].iloc[-2], color='green')
    plt.bar(profile['price'], profile['sell_volume'],
            width=profile['price'].iloc[-1] - profile['price'].iloc[-2], color='red', bottom=profile['buy_volume'])
    plt.show()

In [ ]:
btc_1m = kline_data(interval='1m')
d_hl_p = (btc_1m['high'] - btc_1m['low']) / btc_1m['open']
print(f'Mean price percent change in 1m candles: {np.mean(d_hl_p)}')
print(f'Std price percent change in 1m candles: {np.std(d_hl_p)}')
print(f'Mean + 2 Std: {np.mean(d_hl_p) + 2 * np.std(d_hl_p)}')

Last saved data candle close time: 2021-08-18 14:50:00


We can use only close price on 1m chart for obtaining volume profile.

In [ ]:
profile = get_daily_volume_profile(btc_1m, '2021/8/7', n_bins=50)
plot_volume_profile(profile)